[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klar74/WS2025_lecture/blob/main/Vorlesung_28/VL28_RAG_Demo.ipynb)

# VL28 Notebook 3: RAG Demo
## Retrieval-Augmented Generation (ohne API-Keys!)

In diesem Notebook lernen wir:
- Was RAG ist: LLM + externe Dokumente
- Wie man eine einfache Vektor-Datenbank baut
- Semantic Search mit Sentence-Transformers
- Warum RAG Halluzinationen reduziert
- **Alles lokal, kein OpenAI/API-Key nötig!**

In [ ]:
# Installation (nur einmal ausführen)
# !pip install sentence-transformers numpy scikit-learn

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

## 1. Das Problem: LLMs haben begrenztes Wissen

**Beispiel-Szenario:** Firmen-Handbuch mit internen Regeln
- LLM kennt diese Dokumente nicht (waren nicht im Training)
- LLM könnte Antworten "erfinden" (Halluzination)

**RAG-Lösung:** 
1. **Retrieval**: Finde relevante Dokumente
2. **Augmentation**: Füge Dokumente zum Prompt hinzu
3. **Generation**: LLM antwortet basierend auf Dokumenten

## 2. Schritt 1: Dokumente vorbereiten

Wir simulieren ein Firmen-Handbuch mit Urlaubsregeln, Arbeitszeiten, etc.

In [ ]:
# Beispiel-Dokumente (Firmen-Handbuch)
dokumente = [
    {
        "id": "DOC001",
        "titel": "Urlaubsregelung",
        "text": "Mitarbeiter haben Anspruch auf 30 Tage Urlaub pro Jahr. Urlaub muss mindestens 2 Wochen im Voraus beantragt werden."
    },
    {
        "id": "DOC002", 
        "titel": "Arbeitszeiten",
        "text": "Die reguläre Arbeitszeit beträgt 40 Stunden pro Woche, Montag bis Freitag von 9:00 bis 17:00 Uhr. Gleitzeit ist möglich zwischen 7:00 und 19:00 Uhr."
    },
    {
        "id": "DOC003",
        "titel": "Homeoffice-Regelung", 
        "text": "Mitarbeiter können bis zu 3 Tage pro Woche im Homeoffice arbeiten. Dienstags ist Präsenzpflicht für Team-Meetings."
    },
    {
        "id": "DOC004",
        "titel": "Krankheit",
        "text": "Bei Krankheit muss der Vorgesetzte am ersten Tag telefonisch informiert werden. Ab dem 3. Tag ist ein ärztliches Attest erforderlich."
    },
    {
        "id": "DOC005",
        "titel": "Weiterbildung",
        "text": "Jeder Mitarbeiter hat ein jährliches Weiterbildungsbudget von 2000 Euro. Schulungen müssen mit der Personalabteilung abgestimmt werden."
    }
]

print(f"Anzahl Dokumente: {len(dokumente)}")
for doc in dokumente:
    print(f"  - {doc['id']}: {doc['titel']}")

## 3. Schritt 2: Embeddings erstellen (Vektor-Datenbank)

Wir nutzen **Sentence-Transformers** um jeden Dokumententext in einen Vektor umzuwandeln.

In [ ]:
# Deutsches Sentence-Transformer Modell laden
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
print("✓ Embedding-Modell geladen!")

# Alle Dokumente in Vektoren umwandeln
doc_texte = [doc["text"] for doc in dokumente]
doc_embeddings = model.encode(doc_texte)

print(f"\nEmbeddings erstellt:")
print(f"  Shape: {doc_embeddings.shape}")
print(f"  (5 Dokumente, jeweils 384-dimensionaler Vektor)")

## 4. Schritt 3: Semantic Search (Retrieval)

Wenn ein User eine Frage stellt, suchen wir die relevantesten Dokumente.

In [ ]:
def suche_dokumente(frage, top_k=2):
    """
    Findet die relevantesten Dokumente für eine Frage.
    
    Args:
        frage: User-Frage als String
        top_k: Anzahl der zurückzugebenden Dokumente
    
    Returns:
        Liste der relevantesten Dokumente
    """
    # Frage in Vektor umwandeln
    frage_embedding = model.encode([frage])
    
    # Ähnlichkeit zu allen Dokumenten berechnen
    similarities = cosine_similarity(frage_embedding, doc_embeddings)[0]
    
    # Top K Indizes finden
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    
    # Ergebnisse zusammenstellen
    results = []
    for idx in top_indices:
        results.append({
            "dokument": dokumente[idx],
            "relevanz": similarities[idx]
        })
    
    return results

# Test: Verschiedene Fragen
fragen = [
    "Wie viele Urlaubstage habe ich?",
    "Kann ich von zu Hause arbeiten?",
    "Was muss ich bei Krankheit tun?",
    "Wie viel Geld gibt es für Fortbildung?"
]

for frage in fragen:
    print(f"\n{'='*60}")
    print(f"Frage: {frage}")
    print(f"{'='*60}")
    
    results = suche_dokumente(frage, top_k=2)
    
    for i, result in enumerate(results, 1):
        doc = result["dokument"]
        relevanz = result["relevanz"]
        print(f"\n{i}. {doc['titel']} (Relevanz: {relevanz:.3f})")
        print(f"   {doc['text']}")

**Beobachtung:**
- Semantic Search findet Dokumente basierend auf **Bedeutung**, nicht nur Keywords
- "Urlaubstage" findet "Urlaubsregelung" (obwohl "Tage" nicht im Titel steht)
- "von zu Hause arbeiten" findet "Homeoffice-Regelung"

## 5. Schritt 4: RAG-Prompt konstruieren

Jetzt bauen wir einen Prompt, der dem LLM die relevanten Dokumente gibt.

In [ ]:
def erstelle_rag_prompt(frage, top_k=2):
    """
    Erstellt einen RAG-Prompt mit Kontext-Dokumenten.
    """
    # Relevante Dokumente finden
    results = suche_dokumente(frage, top_k)
    
    # Kontext zusammenbauen
    kontext = "\n\n".join([
        f"Dokument {i+1} ({r['dokument']['titel']}):\n{r['dokument']['text']}"
        for i, r in enumerate(results)
    ])
    
    # Prompt konstruieren
    prompt = f"""Du bist ein hilfreicher Assistent, der Fragen basierend auf Firmen-Dokumenten beantwortet.

WICHTIG: Beantworte die Frage NUR basierend auf den folgenden Dokumenten. 
Wenn die Antwort nicht in den Dokumenten steht, sage das deutlich.

KONTEXT:
{kontext}

FRAGE: {frage}

ANTWORT:"""
    
    return prompt, results

# Beispiel
frage = "Wie viele Tage Urlaub stehen mir zu?"
prompt, docs = erstelle_rag_prompt(frage)

print("RAG-Prompt:")
print("="*60)
print(prompt)
print("="*60)
print(f"\nVerwendete Dokumente:")
for doc in docs:
    print(f"  - {doc['dokument']['titel']} (Relevanz: {doc['relevanz']:.3f})")

## 6. Simulation: Mit vs. Ohne RAG

**Ohne RAG:** LLM könnte halluzinieren oder veraltete Infos geben

**Mit RAG:** LLM hat die echten Dokumente und kann daraus zitieren

In [ ]:
# Weitere Beispiele
test_fragen = [
    "Muss ich bei Krankheit ein Attest vorlegen?",
    "Wann muss ich ins Büro kommen?",
    "Wie hoch ist mein Weiterbildungsbudget?",
]

for frage in test_fragen:
    print(f"\n{'#'*60}")
    print(f"FRAGE: {frage}")
    print(f"{'#'*60}")
    
    prompt, docs = erstelle_rag_prompt(frage, top_k=1)
    
    print("\n🔍 Gefundene Dokumente:")
    for doc in docs:
        print(f"   → {doc['dokument']['titel']} (Relevanz: {doc['relevanz']:.3f})")
        print(f"     {doc['dokument']['text'][:100]}...")
    
    print("\n✅ ERWARTETE ANTWORT (basierend auf Dokument):")
    # Hier würde normalerweise ein LLM antworten
    # Wir simulieren die ideale Antwort
    if "Attest" in frage:
        print("   Ja, ab dem 3. Krankheitstag ist ein ärztliches Attest erforderlich.")
    elif "Büro" in frage or "ins Büro" in frage.lower():
        print("   Dienstags besteht Präsenzpflicht für Team-Meetings. Ansonsten ist Homeoffice möglich.")
    elif "Weiterbildung" in frage or "Budget" in frage:
        print("   Das jährliche Weiterbildungsbudget beträgt 2000 Euro pro Mitarbeiter.")

## 7. Vorteile von RAG visualisieren

Vergleich: Wie relevant sind die gefundenen Dokumente?

In [ ]:
import matplotlib.pyplot as plt

# Alle Fragen durchgehen und Relevanz-Scores sammeln
fragen_test = [
    "Urlaubstage",
    "Homeoffice", 
    "Krankheit",
    "Weiterbildung",
    "Arbeitszeit"
]

fig, axes = plt.subplots(1, len(fragen_test), figsize=(15, 3))

for i, frage in enumerate(fragen_test):
    results = suche_dokumente(frage, top_k=len(dokumente))
    relevanz_scores = [r["relevanz"] for r in results]
    titel = [r["dokument"]["titel"][:15] for r in results]
    
    axes[i].barh(range(len(titel)), relevanz_scores, color='steelblue')
    axes[i].set_yticks(range(len(titel)))
    axes[i].set_yticklabels(titel, fontsize=8)
    axes[i].set_xlabel('Relevanz', fontsize=8)
    axes[i].set_title(frage, fontsize=10)
    axes[i].set_xlim(0, 1)

plt.suptitle("Retrieval-Qualität: Semantic Search findet relevante Dokumente", fontsize=12)
plt.tight_layout()
plt.show()

## 🎯 Takeaway

**RAG löst fundamentale LLM-Probleme:**
- ✅ **Kein veraltetes Wissen**: Dokumente sind immer aktuell
- ✅ **Weniger Halluzinationen**: LLM antwortet basierend auf echten Quellen
- ✅ **Firmen-spezifisches Wissen**: Interne Dokumente, die nie öffentlich waren
- ✅ **Nachvollziehbarkeit**: Man sieht, welche Dokumente verwendet wurden

**Wie RAG funktioniert:**
1. **Embedding-Modell** wandelt Dokumente in Vektoren um (Vektor-DB)
2. **Semantic Search** findet relevante Dokumente zur Frage
3. **Prompt-Augmentation** fügt Dokumente zum Kontext hinzu
4. **LLM** generiert Antwort basierend auf Dokumenten

**RAG in der Praxis:**
- Kundensupport-Bots (Produkthandbücher)
- Firmen-Wikis und Dokumentensuche
- Juristische/Medizinische Q&A (Compliance!)
- Code-Assistenten (eigener Codebase als Kontext)

**Wichtig:** RAG ersetzt kein echtes Verständnis, aber es macht LLMs **verlässlicher und nützlicher**!

---

## 🎓 Ende der VL28 Notebooks

**Was wir gelernt haben:**
1. **Word Embeddings**: Bedeutung durch Vektorraum (aber statisch)
2. **BERT**: Kontextabhängige Embeddings (Bank = Geldinstitut vs. Sitzmöbel)
3. **RAG**: LLMs mit externem Wissen erweitern

**Alles ohne API-Keys!** Hugging Face + Sentence-Transformers = kostenlos und lokal nutzbar.